# 🚀 Module 3: Model Packaging and Deployment on Kubernetes

In this module, we will:
1. Build a REST API for model inference using FastAPI
2. Create a Containerfile to containerize the service
3. Deploy the container to a Kubernetes cluster (i.e. OpenShift)
4. Load a Test Dataset
5. Test the Model-API

## 🛠️ Create a REST API using FastAPI
This API will load the model and expose an endpoint for predictions.

In [ ]:
%%writefile ./models/app.py
# ./models/app.py
import os
from fastapi import FastAPI, HTTPException
import pandas as pd
import mlflow
from mlflow.pyfunc import PyFuncModel

# ── 1. Read configuration from environment ──────────────────────────────
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI") # e.g. "https://mlflow_tracking_server.com"
MODEL_NAME = os.getenv("MODEL_NAME") # e.g. "BikeSharingModel" 
MODEL_VERSION = os.getenv("MODEL_VERSION") # e.g. "5" 

if not MLFLOW_TRACKING_URI:
    raise RuntimeError("MLFLOW_TRACKING_URI environment variable not set")

if not MODEL_VERSION:
    raise RuntimeError("MODEL_VERSION environment variable not set!")

# ── 2. Connect to MLflow and load the model once at startup ─────────────
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# load a specific version
model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"

model: PyFuncModel = mlflow.pyfunc.load_model(model_uri)

# ── 3. API definition ───────────────────────────────────────────────────
app = FastAPI(title="Bike-Sharing Predictor",
              description=f"Served from {model_uri} at {MLFLOW_TRACKING_URI}",
              version="1.0.0")

# Health check model
class HealthCheck(BaseModel):
    status: str = "OK"

@app.get("/health", response_model=HealthCheck, status_code=status.HTTP_200_OK,
         summary="Health check endpoint")
def health_check():
    return HealthCheck(status="OK")

@app.get("/", include_in_schema=False, summary="Root welcome or redirect")
def root():
    # Option A: friendly message
    return {"message": "Hello! Please try /docs to see the available endpoints."}

@app.post("/predict")
def predict(features: dict):
    """
    Accepts a JSON object of feature names / values
    and returns a single prediction.
    """
    try:
        df = pd.DataFrame([features])
        prediction = float(model.predict(df)[0])  # ensure JSON-serialisable
        return {"prediction": prediction}
    except Exception as exc:
        raise HTTPException(status_code=400, detail=str(exc))

Overwriting ./models/app.py


## 📦 Containerize the FastAPI Application
Create a Containerfile for the FastAPI app.

In [ ]:
%%writefile ./models/Containerfile
# ./models/Containerfile
FROM python:3.11-slim

# ── Install OS dependencies (optional but helpful) ───────────────────────
RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential         \
    && rm -rf /var/lib/apt/lists/*

# ── Set workdir and copy application code ────────────────────────────────
WORKDIR /app
COPY app.py ./

# ── Install Python requirements ──────────────────────────────────────────
# mlflow pulls in scikit-learn, pandas, etc.  --no-cache-dir keeps image small
RUN pip install --no-cache-dir fastapi uvicorn[standard] mlflow pandas

# ── Environment variables with sensible defaults (override at runtime) ──

# ── MLflow Tracking URI ────────────────────────────────────────────────
# This is the URI of the MLflow Tracking Server where the model is registered
# e.g. "https://mlflow_tracking_server.com"
ENV MLFLOW_TRACKING_URI=""

# ── Model name ───────────────────────────────────────────── 
# This is the name of the model registered in MLflow
# e.g. "BikeSharingModel"
ENV MODEL_NAME=""

# ── Model version ────────────────────────────────────────────
# This is the version of the model to load from MLflow.
# e.g. "1"
ENV MODEL_VERSION=""

# ── Entrypoint ───────────────────────────────────────────────────────────
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

Overwriting ./models/Containerfile


#### Congratulations! You have completed all the steps in task 5 (`Model Deploymet - Containerize the Endpoint-API`). 
#### Please go back to the instructions on the GitHub-Pages.

#### We will continue with the steps in task 6 (`Model Deploymet - Deploy on OpenShift Cluster`). 

## 🚀 Deployment on OpenShift Cluster: Test the Model-API
In this section, we deploy the trained machine learning model on an OpenShift cluster. The deployment includes deploying the image via a deployment manifest and exposing it using an internal service.

- Deploy the Image Using k8s_deployment.yaml
- Expose the Model Internally via a Service
- Expose the Model Externally via a Route (Optional)